In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True

In [2]:
# because the kernel will die
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

##### augment and load data

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # This will normalize the image in the range [-1,1]
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}

data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### model training 
a general function to train a model. Here, we will illustrate:

- Scheduling the learning rate
- Saving the best model

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

##### 1-fine tuning ResNet18 pretrained model

load the pretrained ResNet18 model and reset final fully connected layer and retrain the whole model on the new dataset

In [6]:
%%time
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6623 Acc: 0.6885
val Loss: 0.2552 Acc: 0.9281

Epoch 1/24
----------
train Loss: 0.3991 Acc: 0.8320
val Loss: 0.2808 Acc: 0.8889

Epoch 2/24
----------
train Loss: 0.5798 Acc: 0.7787
val Loss: 0.5907 Acc: 0.8170

Epoch 3/24
----------
train Loss: 0.5036 Acc: 0.8361
val Loss: 0.4666 Acc: 0.8235

Epoch 4/24
----------
train Loss: 0.5441 Acc: 0.7746
val Loss: 0.6370 Acc: 0.8039

Epoch 5/24
----------
train Loss: 0.5591 Acc: 0.7787
val Loss: 1.0716 Acc: 0.6732

Epoch 6/24
----------
train Loss: 0.5192 Acc: 0.8033
val Loss: 0.5516 Acc: 0.8105

Epoch 7/24
----------
train Loss: 0.4700 Acc: 0.8402
val Loss: 0.3506 Acc: 0.8627

Epoch 8/24
----------
train Loss: 0.3795 Acc: 0.8484
val Loss: 0.3729 Acc: 0.8824

Epoch 9/24
----------
train Loss: 0.2905 Acc: 0.8770
val Loss: 0.2604 Acc: 0.8824

Epoch 10/24
----------
train Loss: 0.3146 Acc: 0.8934
val Loss: 0.2668 Acc: 0.9020

Epoch 11/24
----------
train Loss: 0.4618 Acc: 0.8361
val Loss: 0.2677 Acc: 0.8954

Ep

##### use the pretrained ResNet18 as a feature extractor

we need to freeze all the network except the final layer

In [8]:
%%time
# We need to set requires_grad == False to freeze the parameters 
# so that the gradients are not computed in backward()
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as opposed to 
# the previous step
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5530 Acc: 0.6885
val Loss: 0.2405 Acc: 0.9085

Epoch 1/24
----------
train Loss: 0.4833 Acc: 0.7664
val Loss: 0.2109 Acc: 0.9216

Epoch 2/24
----------
train Loss: 0.4583 Acc: 0.7828
val Loss: 0.2068 Acc: 0.9085

Epoch 3/24
----------
train Loss: 0.4028 Acc: 0.8402
val Loss: 0.2031 Acc: 0.9281

Epoch 4/24
----------
train Loss: 0.5273 Acc: 0.7664
val Loss: 0.3995 Acc: 0.8366

Epoch 5/24
----------
train Loss: 0.3953 Acc: 0.8279
val Loss: 0.1988 Acc: 0.9346

Epoch 6/24
----------
train Loss: 0.7564 Acc: 0.7254
val Loss: 0.2103 Acc: 0.9412

Epoch 7/24
----------
train Loss: 0.3413 Acc: 0.8402
val Loss: 0.2168 Acc: 0.9085

Epoch 8/24
----------
train Loss: 0.3778 Acc: 0.8607
val Loss: 0.1523 Acc: 0.9542

Epoch 9/24
----------
train Loss: 0.4429 Acc: 0.7992
val Loss: 0.2358 Acc: 0.9216

Epoch 10/24
----------
train Loss: 0.3892 Acc: 0.8279
val Loss: 0.2505 Acc: 0.9150

Epoch 11/24
----------
train Loss: 0.3791 Acc: 0.8361
val Loss: 0.1639 Acc: 0.9608

Ep